<a href="https://colab.research.google.com/github/vuongthutrang174/CNN/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np # Xử lý một số toán học
import matplotlib.pyplot as plt # Thư viện đồ thị trực quan, hình ảnh,...
from pathlib import Path
import os, sys
import cv2 
import random
import pickle
import tensorflow as tf
from tensorflow.data import AUTOTUNE
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = "/content/drive/MyDrive/Dateset_W3"
data_dir = Path(data_dir)
categories = os.listdir(data_dir) 
number_classes = len(categories)
print(categories)

['Benign', 'InSitu', 'Invasive', 'Normal']


In [ ]:
batch_size = 32
img_height = 1536
img_width = 2048
scale_percent = 25
width = int(img_width * scale_percent / 100)
height = int(img_height * scale_percent / 100)
dim = (width, height)


In [ ]:
X = []
y = []

In [ ]:

new_img_array = None
def create_training_data():
  for category in categories: 

    path = os.path.join(data_dir, category)  
    print(path)
    label_number = categories.index(category) 
    for img in os.listdir(path): 
      try:
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_ANYCOLOR) 
        new_img_array = cv2.resize(img_array, dim, interpolation = cv2.INTER_AREA) 
        new_img_array = np.array(new_img_array)
        X.append(new_img_array)  
        y.append(label_number)  
      except Exception as e:
        pass
    print(category + " success!")
    
create_training_data()


/content/drive/MyDrive/Dateset_W3/Benign
Benign success!
/content/drive/MyDrive/Dateset_W3/InSitu
InSitu success!
/content/drive/MyDrive/Dateset_W3/Invasive
Invasive success!
/content/drive/MyDrive/Dateset_W3/Normal
Normal success!


In [ ]:
# # Chạy 1 lần, thành công thì không chạy nữa
pickle_out = open("/content/drive/MyDrive/W3_CNN/X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [ ]:
# # Chạy 1 lần, thành công thì không chạy nữa
pickle_out = open("/content/drive/MyDrive/W3_CNN/y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
# Mỗi lần chạy mô hình thì chạy để load dữ liệu
pickle_in = open("/content/drive/MyDrive/W3_CNN/X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("/content/drive/MyDrive/W3_CNN/y.pickle", "rb")
y = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
X = np.array(X)

In [ ]:
X = X / 255

In [ ]:

X.shape

(1159, 384, 512, 3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
X = y = []
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=0.2, random_state=42)
y_train = to_categorical(y_train) # one hot
# ví dụ y_number = 1 thì y = [0. 1. 0.]
y_t = y_test
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [ ]:
model = Sequential() 

model.add(Conv2D(filters=16, kernel_size=(3, 3), input_shape=X_train.shape[1:], padding="same", activation="relu"))
model.add(AveragePooling2D(pool_size=(4, 4), strides=2))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="tanh"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="selu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="selu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(number_classes))
model.add(Activation("softmax"))

# Cung cấp cho mô hình hàm loss function lựa chọn, thuật toán tối ưu
# categorical_crossentropy, Binary Cross Entropy
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary() # xem chi tiết mô hình xây dựng của chính mình

# Cuối cùng đào tạo nó
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=25)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 384, 512, 16)      448       
_________________________________________________________________
average_pooling2d (AveragePo (None, 191, 255, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 189, 253, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 94, 126, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 92, 124, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 44, 60, 64)        3

In [ ]:
cd "/content/drive/MyDrive/W3_CNN"


/content/drive/MyDrive/W3_CNN


In [ ]:
mkdir -p saved_model


In [ ]:
model.save('saved_model/my_model')
new_model = load_model('saved_model/my_model')

new_model.summary()

INFO:tensorflow:Assets written to: saved_model/my_model/assets
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 384, 512, 16)      448       
_________________________________________________________________
average_pooling2d (AveragePo (None, 191, 255, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 189, 253, 32)      4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 94, 126, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 92, 124, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 62, 64)        0         
___________________________________________________________

In [ ]:
loss, acc = new_model.evaluate(X_test, y_test, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

8/8 - 15s - loss: 1.4936 - accuracy: 0.5905
Restored model, accuracy: 59.05%
